In [42]:
import os
import torch
import csv

from pathlib import Path
import io
import lmdb
import pickle
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
from torch.utils.data import Dataset
from torchvision.transforms.functional import to_tensor, to_pil_image
import torch.nn.functional as F

from nuscenes.nuscenes import NuScenes
from nuscenes.utils.geometry_utils import (
    view_points,
    box_in_image,
    BoxVisibility,
    transform_matrix,
)
from nuscenes.map_expansion.map_api import NuScenesMap, NuScenesMapExplorer
from nuscenes.utils.data_classes import LidarPointCloud



In [43]:
root=os.path.join("nuscenes_data")
nusc = NuScenes(version="v1.0-mini",
    dataroot=root,
    verbose=False)
nusc

In [ ]:
def read_split(filename):
    """
    Read a list of NuScenes sample tokens
    """
    with open(filename, "r") as f:
        lines = f.read().split("\n")
        return [val for val in lines if val != ""]


In [ ]:
tokens = read_split(
    os.path.join(root, "splits", "{}.txt".format("train_full"))
)


In [ ]:
root

In [44]:
images_db_path = os.path.join(
            root, "lmdb",
            "samples"
        )
images_db_path

'nuscenes_data\\lmdb\\samples'

In [45]:
images_db = lmdb.open(
    path=images_db_path,
    readonly=True,
    readahead=False,
    max_spare_txns=128,
    lock=False,
)

images_db


In [107]:
sample_token = '35833ae5808e4ef186d1fdebac3d9cf6'
sample_record = nusc.get("sample", sample_token)
cam_token = sample_record["data"]["CAM_FRONT"]
cam_record = nusc.get("sample_data", cam_token)
cam_path = nusc.get_sample_data_path(cam_token)
id = Path(cam_path).stem
# Load intrinsincs
calib = nusc.get(
    "calibrated_sensor", cam_record["calibrated_sensor_token"]
)["camera_intrinsic"]
calib = np.array(calib)

# Load input images

image_input_key = pickle.dumps(id ,protocol=3)
image_input_key

b'\x80\x03X:\x00\x00\x00n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385111912404q\x00.'

In [108]:
with images_db.begin() as txn:
    value = txn.get(key=image_input_key)
    image = Image.open(io.BytesIO(value)).convert(mode='RGB')
    

In [109]:
image.show()

In [94]:
gtmaps_db_path = os.path.join(
            root, "semantic_maps_new_200x200",
)

gtmaps_db_path

lst = os.listdir(gtmaps_db_path) # your directory path
lst


['data.mdb', 'fov_mask.png', 'lock.mdb']

In [95]:
gtmaps_db = lmdb.open(
            path=gtmaps_db_path,
            readonly=True,
            readahead=False,
            max_spare_txns=128,
            lock=False,
        )

gtmaps_db

In [96]:
classes = [
            "drivable_area",
            "ped_crossing",
            "walkway",
            "carpark_area",
            "road_segment",
            "lane",
            "bus",
            "bicycle",
            "car",
            "construction_vehicle",
            "motorcycle",
            "trailer",
            "truck",
            "pedestrian",
            "trafficcone",
            "barrier",
        ] + ['lidar_ray_mask_dense']
len(classes)

17

In [97]:
fov_mask = to_tensor(Image.open(
            os.path.join(root, "semantic_maps_new_200x200", "fov_mask.png")
        ))

In [98]:
# i=0
# cam_id = token_cam_id_map[tokens[i]]
# cam_id

In [99]:
def merge_map_classes(mapsdict):
    classes_to_merge = ["drivable_area", "road_segment", "lane"]
    merged_class = 'drivable_area'
    maps2merge = torch.stack([mapsdict[k] for k in classes_to_merge])  # [n, 1, 200, 200]
    maps2merge = maps2merge.sum(dim=0)
    maps2merge = (maps2merge > 0).float()
    mapsdict[merged_class] = maps2merge
    del mapsdict['road_segment'], mapsdict['lane']
    return mapsdict

In [80]:
def get_ground_truth_pixel_count(cam_id):
    
    gtmaps_key = [pickle.dumps("{}___{}".format(cam_id, cls), 3) for cls in classes]
    with gtmaps_db.begin() as txn:
        value = [txn.get(key=key) for key in gtmaps_key]
        gtmaps = [Image.open(io.BytesIO(im)) for im in value]
    
    # each map is of shape [1, 200, 200]
    mapsdict = {cls: to_tensor(map) for cls, map in zip(classes, gtmaps)}
    # mapsdict["fov_mask"] = to_tensor(fov_mask)
    mapsdict = merge_map_classes(mapsdict)

    # Create visbility mask from lidar and fov masks
    lidar_ray_mask = mapsdict['lidar_ray_mask_dense']
    # fov_mask = mapsdict['fov_mask']
    vis_mask = lidar_ray_mask * fov_mask
    mapsdict['vis_mask'] = vis_mask

    del mapsdict['lidar_ray_mask_dense']#, mapsdict['fov_mask']
    # apply vis mask to maps
    mapsdict = {
        cls: cls_map * mapsdict['vis_mask'] for cls, cls_map in mapsdict.items()
    }
    
    # del mapsdict['vis_mask']
    
    return {cls: cls_map.numpy().sum() for cls, cls_map in mapsdict.items()}

In [115]:
gtmaps_key = [pickle.dumps("{}___{}".format(id, cls), 3) for cls in classes]
with gtmaps_db.begin() as txn:
    value = [txn.get(key=key) for key in gtmaps_key]
    gtmaps = Image.open(io.BytesIO(value[2]))

gtmaps.size

(200, 200)

In [ ]:
with images_db.begin() as txn:
    value = txn.get(key=image_input_key)
    image = Image.open(io.BytesIO(value)).convert(mode='RGB')
    

In [81]:
get_ground_truth_pixel_count(id)


{'drivable_area': 10417.0,
 'ped_crossing': 1492.0,
 'walkway': 4683.0,
 'carpark_area': 933.0,
 'bus': 0.0,
 'bicycle': 0.0,
 'car': 564.0,
 'construction_vehicle': 0.0,
 'motorcycle': 0.0,
 'trailer': 0.0,
 'truck': 0.0,
 'pedestrian': 180.0,
 'trafficcone': 0.0,
 'barrier': 0.0,
 'vis_mask': 16107.0}

In [82]:
train_pixel_counts = {token: get_ground_truth_pixel_count(token_cam_id_map[token]) for token in tokens}


NameError: name 'tokens' is not defined

In [37]:
sample_token = 'e0845f5322254dafadbbed75aaa07969'
sample_record = nusc.get("sample", sample_token)
cam_token = sample_record["data"]["CAM_FRONT"]
cam_record = nusc.get("sample_data", cam_token)
cam_path = nusc.get_sample_data_path(cam_token)
id = Path(cam_path).stem
image_input_key = pickle.dumps(id ,protocol=3)
with images_db.begin() as txn:
    value = txn.get(key=image_input_key)
    image = Image.open(io.BytesIO(value)).convert(mode='RGB')

image.show()

In [21]:
original_nusenes_dir = Path.resolve(Path('C:/Users/Admin/Desktop/translating-images-into-maps/nuscenes_data/lmdb/samples_full/CAM_FRONT/n008-2018-05-21-11-06-59-0400__CAM_FRONT__1526915246012465.jpg'))
image = Image.open(original_nusenes_dir).convert(mode='RGB')
image.show()

image.size

(1600, 900)

In [22]:
og_w, og_h = 1600, 900
desired_image_size=(480, 270)

desired_w, desired_h = desired_image_size

scale_w, scale_h = desired_w / og_w, desired_h / og_h

b = image.resize((int(image.size[0] * scale_w), int(image.size[1] * scale_h)))
a = to_tensor(b)
print(a.shape)


torch.Size([3, 270, 480])


In [ ]:
from difflib import SequenceMatcher

SequenceMatcher(None, 'samples/CAM_TOP/n008-2018-08-01-15-16-36-0400__CAM_TOP__1533151621412404.jpg', "samples/CAM_FRONT/").ratio()

In [11]:
gtmaps_db_path = os.path.join(
            root, "semantic_maps_new_200x200",
)

gtmaps_db_path

lst = os.listdir(gtmaps_db_path) # your directory path
lst


['data.mdb', 'lock.mdb']

In [12]:
gtmaps_db = lmdb.open(
            path=gtmaps_db_path,
            readonly=True,
            readahead=False,
            max_spare_txns=128,
            lock=False,
        )

gtmaps_db

In [27]:
classes=[
            "drivable_area",
            "ped_crossing",
            "walkway",
            "carpark_area",
            "road_segment",
            "lane",
            "bus",
            "bicycle",
            "car",
            "construction_vehicle",
            "motorcycle",
            "trailer",
            "truck",
            "pedestrian",
            "trafficcone",
            "barrier",
        ],
classes = list(classes)
classes.append("lidar_ray_mask_dense")

gtmaps_key = [pickle.dumps("{}___{}".format(id, cls),protocol=3) for cls in classes]

with gtmaps_db.begin() as txn:
    value = [txn.get(key=key) for key in gtmaps_key]
    print(value)
    gtmaps = [Image.open(io.BytesIO(im)) for im in value]
    print(gtmaps)
    

[None, b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xc8\x00\x00\x00\xc8\x08\x00\x00\x00\x00\x883\xf1B\x00\x00\x08\xebIDATx\x9c\xed]\xdb\xb6\xe4*\x08\x94\xfe\xff\x7ff?D\r\x08x\xc5\xe8\xc3f\xcd\x99N\xd2F)\xabP4\x99>!|`\x18p{\x1b\xb0\xbd\x85\x10\xf0\x8bf~\xdb[\xf8\xc8>\x00\x82\xf1\xbf\xbd\xb6\x1f\x08\x92\xbf7\xda\xbf\xb4z\r\x8b\xcf]\xb6\x1b\x08*G[\xec_Z}\x86\xc6\xb1\xbf\xed\x05\xb2\x7f\xd4\xcd\xf6\xa5\xb4\xb6\xc2\xda\n\xa4\xf4|\'\x92\x9d@>\x14\xd6\xd7\xa3\xd6Fh\x1b\x81|J\xc8\xd7\xf3\xc8>p\xfb\x80\xe8>oC\xb2\r\xc8\xb7\xc2:\x90\xa2\xec\x02\xb8\x0b\x88\xed\xef&$\xffIc\xddj\xdd\xbe\x87\x92\x7fF\xaaV\xef\xf4-\x94\x1cad\x07\x92-@\xbe\x9eCB\xd8\x03\xa4\x8dc\x03\xd2C\xc1\xee\x8fd\x03\x90\x13\xc2:7\xfc\xba\xa3\xf5\x07r\x86\x10\x7f \xbd8\xbc\xf1\x9e\x9b\xd9\x9d\x91x\x039$,w #8|1\x9fL\x1a]\x91\xf8\x029&\xac\xc3i\xbc\'nW \x07\tq\x052\x8e\xc3\x11\xf9\xe1\x15\xa2\x1f\x12G \'\x85u\x9c\x11?\xf4~@&]\xf2B\xe2\x06\xe4\xac\xb0\xceK\xcb\xad\x07\xbc\x80\x9c&\xe4\x02F\x9c\xfa\xc0\t\xc8qB\x9c\x80\xac\xe1p\x

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000002894127A9F0>

In [22]:
with gtmaps_db.begin() as txn:
    value = [txn.get(key=key) for key in gtmaps_key]
    print(value)
    gtmaps = [Image.open(io.BytesIO(im)) for im in value]
    print(gtmaps)

[None, None]


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000002894127AEF0>